LOG WRITER DIMENSIONS

To look at full dataframe -> print(df) <br>
To look at seperated dataframe -> print(dfs)/print(dict['df_(dataframe number)']) <br>
To look at rank scores -> print(rank_dataframe)

In [1]:
# WRITING THE LOG
import pandas as pd

file_format = ['csv', 'avro', 'parquet', 'orc']
schemas = ['st','vt', 'pt', 'extvt', 'wpt']
partition = ['horizontal','predicate', 'subject']
dimension = 'storage'
size = 250

# print('Dimension Type:')
# dimension = input()

# # take dataset size
# print("Storage size: ")
# size = input()

# file_format = []
# schemas = []
# partition = []

# # take file format input
# print("File Format: ")
# while True:
#     line = input()
#     if line:
#         file_format.append(line)
#     else:
#         break

# # take schemas input
# print("Schemas: ")
# while True:
#     line = input()
#     if line:
#         schemas.append(line)
#     else:
#         break

# # take partition input
# print("Partition: ")
# while True:
#     line = input()
#     if line:
#         partition.append(line)
#     else:
#         break

if dimension == 'storage': 
    idx = []
    li=[]
    # READ LOG - STORAGE FORMAT
    for i in schemas:
        for j in partition:
            for k in file_format:
                # check folder
                print(f'./logs/{k}/{size}M_{i}_{j}.txt')

                df = pd.read_csv(f'./logs/{k}/{size}M_{i}_{j}.txt',sep = ',', header = None)
                df = df.fillna(0)
                avg = df.mean(axis = 0)
                li.append(avg)
                idx.append(k + f"_{i}_{j}")

    # create dataframe
    df = pd.DataFrame(li, index = [idx])
    df = df.set_axis(["Q"+str(i+1) for i in range(11)], axis = 1)
    df = df.fillna(5000)
    # split dataframe
    dict = {}
    count = 0
    loop = len(file_format)
    dfs = []
    for i in range(int(len(partition) * len(file_format) * len(schemas) / len(file_format))):
        dict['df_{}'.format(i)] = df[count:loop]
        count = loop
        loop = loop+len(file_format)
        dfs.append(dict[f'df_{i}'])

    # CREATE RANK OCCURENCES
    import scipy.stats as ss
    import numpy as np

    rank_dataframe = []

    for x in dfs:
        df_ranks = x.T
        column_names = df_ranks.columns.to_numpy().tolist()
        column_names = [column_names for column_names, in column_names]
        
        df_ranks_occurences = []
        for index, row in df_ranks.iterrows():
            df_ranks_occurences.append(ss.rankdata(row, method = 'max'))

        df_ranks_occurences = pd.DataFrame(df_ranks_occurences)

        df_transpose = df_ranks_occurences.transpose()

        rank_table = []
        for index, row in df_transpose.iterrows():
            result_row = np.zeros(len(df_transpose.index))
            for i in range(len(row)):
                result_row[int(row[i])-1] +=1
            rank_table.append(result_row)

        rank_table = pd.DataFrame(rank_table)
        rank_table = rank_table.set_axis(column_names, axis = 'index')
        rank_table = rank_table.set_axis([i+1 for i in range(len(column_names))], axis='columns')

        # CREATE R SCORE
        q = 11
        d = len(rank_table.index)

        rank_score = []
        for index, row in rank_table.iterrows():
            s = 0
            for r in range(d):  
                s = s + (row[r+1]*(d-(r+1)) / (q*(d-1)) )
            rank_score.append(s)
        rank_score = pd.DataFrame(rank_score)
        rank_score = rank_score.set_axis(column_names, axis = 'index')
        rank_score = rank_score.set_axis(['Result'], axis='columns')
        rank_score = pd.concat([rank_table, rank_score], axis = 1)
        rank_dataframe.append(rank_score)
####################################################################################################
elif dimension == 'schema':
    idx = []
    li=[]
    # READ LOG - STORAGE FORMAT
    for j in partition:
        for k in file_format:
            for i in schemas:
                # check folder
                print(f'./logs/{k}/100M_{i}_{j}.txt')

                df = pd.read_csv(f'./logs/{k}/100M_{i}_{j}.txt',sep = ',', header = None)
                df = df.fillna(0)
                avg = df.mean(axis = 0)
                li.append(avg)
                idx.append(k + f"_{i}_{j}")

    # create dataframe
    df = pd.DataFrame(li, index = [idx])
    df = df.set_axis(["Q"+str(i+1) for i in range(11)], axis = 1)
    df = df.fillna(5000)
    # split dataframe
    dict = {}
    count = 0
    loop = len(schemas)
    dfs = []
    for i in range(int(len(partition) * len(file_format) * len(schemas) / len(schemas))):
        dict['df_{}'.format(i)] = df[count:loop]
        count = loop
        loop = loop+len(schemas)
        dfs.append(dict[f'df_{i}'])

    # CREATE RANK OCCURENCES
    import scipy.stats as ss
    import numpy as np

    rank_dataframe = []

    for x in dfs:
        df_ranks = x.T
        column_names = df_ranks.columns.to_numpy().tolist()
        column_names = [column_names for column_names, in column_names]
        
        df_ranks_occurences = []
        for index, row in df_ranks.iterrows():
            df_ranks_occurences.append(ss.rankdata(row, method = 'max'))

        df_ranks_occurences = pd.DataFrame(df_ranks_occurences)

        df_transpose = df_ranks_occurences.transpose()

        rank_table = []
        for index, row in df_transpose.iterrows():
            result_row = np.zeros(len(df_transpose.index))
            for i in range(len(row)):
                result_row[int(row[i])-1] +=1
            rank_table.append(result_row)

        rank_table = pd.DataFrame(rank_table)
        rank_table = rank_table.set_axis(column_names, axis = 'index')
        rank_table = rank_table.set_axis([i+1 for i in range(len(column_names))], axis='columns')

        # CREATE R SCORE
        q = 11
        d = len(rank_table.index)

        rank_score = []
        for index, row in rank_table.iterrows():
            s = 0
            for r in range(d):  
                s = s + (row[r+1]*(d-(r+1)) / (q*(d-1)) )
            rank_score.append(s)
        rank_score = pd.DataFrame(rank_score)
        rank_score = rank_score.set_axis(column_names, axis = 'index')
        rank_score = rank_score.set_axis(['Result'], axis='columns')
        rank_score = pd.concat([rank_table, rank_score], axis = 1)
        rank_dataframe.append(rank_score)
########################################################################################################
elif dimension == 'partition':
    idx = []
    li=[]
    # READ LOG - STORAGE FORMAT
    for i in schemas:
        for k in file_format:
            for j in partition:
                # check folder
                print(f'./logs/{k}/100M_{i}_{j}.txt')

                df = pd.read_csv(f'./logs/{k}/100M_{i}_{j}.txt',sep = ',', header = None)
                df = df.fillna(0)
                avg = df.mean(axis = 0)
                li.append(avg)
                idx.append(k + f"_{i}_{j}")

    # create dataframe
    df = pd.DataFrame(li, index = [idx])
    df = df.set_axis(["Q"+str(i+1) for i in range(11)], axis = 1)
    df = df.fillna(5000)
    # split dataframe
    dict = {}
    count = 0
    loop = len(partition)
    dfs = []
    for i in range(int(len(partition) * len(file_format) * len(schemas) / len(partition))):
        dict['df_{}'.format(i)] = df[count:loop]
        count = loop
        loop = loop+len(partition)
        dfs.append(dict[f'df_{i}'])

    # CREATE RANK OCCURENCES
    import scipy.stats as ss
    import numpy as np

    rank_dataframe = []

    for x in dfs:
        df_ranks = x.T
        column_names = df_ranks.columns.to_numpy().tolist()
        column_names = [column_names for column_names, in column_names]
        
        df_ranks_occurences = []
        for index, row in df_ranks.iterrows():
            df_ranks_occurences.append(ss.rankdata(row, method = 'max'))

        df_ranks_occurences = pd.DataFrame(df_ranks_occurences)

        df_transpose = df_ranks_occurences.transpose()

        rank_table = []
        for index, row in df_transpose.iterrows():
            result_row = np.zeros(len(df_transpose.index))
            for i in range(len(row)):
                result_row[int(row[i])-1] +=1
            rank_table.append(result_row)

        rank_table = pd.DataFrame(rank_table)
        rank_table = rank_table.set_axis(column_names, axis = 'index')
        rank_table = rank_table.set_axis([i+1 for i in range(len(column_names))], axis='columns')

        # CREATE R SCORE
        q = 11
        d = len(rank_table.index)

        rank_score = []
        for index, row in rank_table.iterrows():
            s = 0
            for r in range(d):  
                s = s + (row[r+1]*(d-(r+1)) / (q*(d-1)) )
            rank_score.append(s)
        rank_score = pd.DataFrame(rank_score)
        rank_score = rank_score.set_axis(column_names, axis = 'index')
        rank_score = rank_score.set_axis(['Result'], axis='columns')
        rank_score = pd.concat([rank_table, rank_score], axis = 1)
        rank_dataframe.append(rank_score)

else:
    print("choose 'storage' or 'partition' or 'schema' dimension")

./logs/csv/250M_st_horizontal.txt
./logs/avro/250M_st_horizontal.txt
./logs/parquet/250M_st_horizontal.txt
./logs/orc/250M_st_horizontal.txt
./logs/csv/250M_st_predicate.txt
./logs/avro/250M_st_predicate.txt
./logs/parquet/250M_st_predicate.txt
./logs/orc/250M_st_predicate.txt
./logs/csv/250M_st_subject.txt
./logs/avro/250M_st_subject.txt
./logs/parquet/250M_st_subject.txt
./logs/orc/250M_st_subject.txt
./logs/csv/250M_vt_horizontal.txt
./logs/avro/250M_vt_horizontal.txt
./logs/parquet/250M_vt_horizontal.txt
./logs/orc/250M_vt_horizontal.txt
./logs/csv/250M_vt_predicate.txt
./logs/avro/250M_vt_predicate.txt
./logs/parquet/250M_vt_predicate.txt
./logs/orc/250M_vt_predicate.txt
./logs/csv/250M_vt_subject.txt
./logs/avro/250M_vt_subject.txt
./logs/parquet/250M_vt_subject.txt
./logs/orc/250M_vt_subject.txt
./logs/csv/250M_pt_horizontal.txt
./logs/avro/250M_pt_horizontal.txt
./logs/parquet/250M_pt_horizontal.txt
./logs/orc/250M_pt_horizontal.txt
./logs/csv/250M_pt_predicate.txt
./logs/avro/

In [2]:
df

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11
csv_st_horizontal,119.811581,434.534540,79.208559,833.553244,252.741987,299.149741,426.360434,537.442719,191.678927,53.058458,35.071883
avro_st_horizontal,61.605208,179.621290,31.063134,658.834816,95.885596,186.678078,288.816110,429.985433,119.866340,29.200286,12.852418
parquet_st_horizontal,39.440573,113.749201,14.500456,591.184875,75.671910,163.391413,245.607044,407.764548,91.364784,7.588160,3.869994
orc_st_horizontal,42.378732,110.825400,13.158544,656.996972,66.485270,203.631652,270.351702,407.240550,94.448321,6.983642,2.788286
csv_st_predicate,116.212925,408.827237,78.534839,823.262067,238.636782,309.673484,409.171700,548.116276,168.274577,82.144783,36.702343
avro_st_predicate,81.318466,164.169820,33.614049,710.471017,104.377956,274.194727,317.947424,485.456736,117.760664,46.507032,21.584215
parquet_st_predicate,54.538511,108.107410,28.664667,643.646836,89.637053,158.227649,268.950900,466.422044,112.448237,14.846991,4.670535
orc_st_predicate,41.796522,98.734800,21.105655,590.733618,72.615040,164.572155,203.416994,448.890928,96.341208,10.637278,2.523962
csv_st_subject,116.935944,422.748822,76.498997,796.847481,246.557554,271.979034,388.947217,527.328294,188.356603,54.592933,34.044350
avro_st_subject,57.643287,169.955491,27.962208,619.322019,89.408717,215.052230,347.350987,426.598422,119.673505,27.230969,12.104282


Save to excel for better view

In [3]:
# Create excel sheet
excel_table = dfs+rank_dataframe
xlwriter = pd.ExcelWriter('~/Desktop/rank_table.xlsx', engine='xlsxwriter')
row = 0
for dataframe in excel_table:
        dataframe.to_excel(xlwriter, sheet_name = 'schema_format_100M', startrow = row, startcol = 0)
        row = row + len(dataframe.index) + 2 
xlwriter.save()

CALCULATING THE CONFORMANCE

df_full_ranks : full dataframe of configurations and their ranks <br>
best_scores :  best 5 configurations from dataframe

In [29]:
# GET FULL RANKS OF DATAFRAME FOR CRITERIA TABLE
df_full_ranks = []
df = df.T
for index, row in df.iterrows():
    df_full_ranks.append(ss.rankdata(row, method = 'max'))

df_full_ranks = pd.DataFrame(df_full_ranks)
df_full_ranks = df_full_ranks.T
df_full_ranks = df_full_ranks.set_axis(df.columns, axis = 'index') #configurations
df_full_ranks = df_full_ranks.set_axis([i+1 for i in range(len(df.index))], axis='columns') #query

In [30]:
# GET BEST SCORES OF 5
full_ranks_scores = pd.concat(rank_dataframe, axis = 0)
full_ranks_scores = full_ranks_scores['Result']
best_scores = full_ranks_scores.nlargest(5)

In [31]:
best_scores

orc_st_subject           0.969697
orc_st_predicate         0.969697
orc_st_horizontal        0.909091
parquet_vt_subject       0.787879
parquet_extvt_subject    0.757576
Name: Result, dtype: float64

In [33]:
best_scores = best_scores.index.to_numpy().tolist()
criteria_table = df_full_ranks.loc[best_scores]
criteria_table

,1,2,3,4,5,6,7,8,9,10,11
orc_st_subject,29,30,21,9,25,38,33,33,30,11,14
orc_st_predicate,36,37,41,30,37,42,34,53,39,25,9
orc_st_horizontal,34,45,30,13,33,43,36,40,33,15,15
parquet_vt_subject,12,25,13,46,8,11,5,34,44,19,2
parquet_extvt_subject,19,9,7,32,11,14,42,19,8,2,60


In [34]:
# GET CONFIGURATION THAT IS HIGHER THAN 30% OF TOTAL DATASET
criteria_table = criteria_table[criteria_table > full_ranks_scores.count() * (30/100)]
criteria_table

,1,2,3,4,5,6,7,8,9,10,11
orc_st_subject,29.0,30.0,21.0,NaN,25.0,38.0,33.0,33,30.0,NaN,NaN
orc_st_predicate,36.0,37.0,41.0,30.0,37.0,42.0,34.0,53,39.0,25.0,NaN
orc_st_horizontal,34.0,45.0,30.0,NaN,33.0,43.0,36.0,40,33.0,NaN,NaN
parquet_vt_subject,NaN,25.0,NaN,46.0,NaN,NaN,NaN,34,44.0,19.0,NaN
parquet_extvt_subject,19.0,NaN,NaN,32.0,NaN,NaN,42.0,19,NaN,NaN,60.0


In [35]:
count = criteria_table.count(axis=1)
avg = count.mean(axis = 0)
count, avg

(orc_st_subject            8
 orc_st_predicate         10
 orc_st_horizontal         8
 parquet_vt_subject        5
 parquet_extvt_subject     5
 dtype: int64,
 7.2)

In [36]:
# GET CONFORMANCE
sum = count.sum(axis = 0)
conformance = 1 - (sum/55)
conformance

0.34545454545454546

CALCULATE COHERENCE

In [3]:
var1 = pd.concat(rank_dataframe, axis = 0)
var1 = var1['Result']

In [7]:
var2 = pd.concat(rank_dataframe, axis = 0)
var2 = var2['Result']

In [11]:
new_df = pd.concat([var1, var2], axis = 1)

In [12]:
column_name = new_df.columns.values

column_name[0] = '100M'
column_name[1] = '250M'

new_df.columns = column_name
new_df

,100M,250M
csv_st_horizontal,0.000000,0.000000
avro_st_horizontal,0.363636,0.363636
parquet_st_horizontal,0.727273,0.818182
orc_st_horizontal,0.909091,0.818182
csv_st_predicate,0.030303,0.000000
avro_st_predicate,0.575758,0.333333
parquet_st_predicate,0.424242,0.696970
orc_st_predicate,0.969697,0.969697
csv_st_subject,0.000000,0.000000
avro_st_subject,0.333333,0.333333


In [13]:
new_df_ranks = new_df.rank(axis = 0, method = 'first', ascending=False)

In [17]:
new_df_ranks = new_df_ranks.sort_values(by = '100M', axis = 0, ascending = True)

In [18]:
# uses kendal's tau correlation coefficient
new_df_ranks.corr(method='kendall')

,100M,250M
100M,1.000000,0.735593
250M,0.735593,1.000000


In [24]:
# use kendall tau b - works with tie variables
ss.kendalltau(var1,var2, nan_policy = 'propagate', method = 'asymptotic', variant = 'b')

KendalltauResult(correlation=0.7457609726798358, pvalue=3.5382912903473637e-16)

In [25]:
# kendall's distance
def normalised_kendall_tau_distance(values1, values2):
    """Compute the Kendall tau distance."""
    n = len(values1)
    assert len(values2) == n, "Both lists have to be of equal length"
    i, j = np.meshgrid(np.arange(n), np.arange(n))
    a = np.argsort(values1)
    b = np.argsort(values2)
    ndisordered = np.logical_or(np.logical_and(a[i] < a[j], b[i] > b[j]), np.logical_and(a[i] > a[j], b[i] < b[j])).sum()
    return ndisordered / (n * (n - 1))

In [32]:
var1_array = var1.to_numpy()
var2_array = var2.to_numpy()

In [33]:
normalised_kendall_tau_distance(var1_array, var2_array)

0.4180790960451977

PARETO CALCULATION - Finding the non-dominated solution from these3 dimensions

In [ ]:
dimensionsAll = df_full_ranks.reset_index().values
dimensionsAll

In [ ]:
dimensions = np.array(df_full_ranks[:], dtype = np.float64)
dimensions

In [ ]:
def nsga2(inputPoints, dominates):
    paretoPoints = set()
    candidateRowNr = 0
    dominatedPoints = set()
    while True:
        candidateRow = inputPoints[candidateRowNr] # masuk row pertama
        inputPoints.remove(candidateRow) # buang row pertamanya
        rowNr = 0
        nonDominated = True # row kandidat yg baru masuk di set true buat non dominated nya
        while len(inputPoints) != 0 and rowNr < len(inputPoints):
            row = inputPoints[rowNr] # ambil row lanjutannya
            if dominates(candidateRow, row): # ngecek row kandidat ama row lanjutan di input_point dominasinya
                # If it is worse on all features remove the row from the array
                inputPoints.remove(row)
                dominatedPoints.add(tuple(row))
            elif dominates(row, candidateRow):
                nonDominated = False
                dominatedPoints.add(tuple(candidateRow))
                rowNr += 1
            else:
                rowNr += 1

        if nonDominated:
            # add the non-dominated point to the Pareto frontier
            paretoPoints.add(tuple(candidateRow))

        if len(inputPoints) == 0:
            break
    return paretoPoints, dominatedPoints

In [ ]:
def dominates(row, candidateRow):
    return np.sum([row[x] <= candidateRow[x] for x in range(len(row))]) == len(row)    # kalo lebih kecil semua berarti dominasi

inputPoints = dimensions.tolist()
type(inputPoints)
paretoPoints, dominatedPoints = nsga2(inputPoints, dominates)

print ("*"*8 + " non-dominated answers " + ("*"*8) )
for p in paretoPoints:
    print (p)
print ("*"*8 + " dominated answers " + ("*"*8))
for p in dominatedPoints:
    print (p)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D



fig = plt.figure()
fig.set_size_inches(15, 15)
ax = fig.add_subplot(111, projection='3d')
dp = np.array(list(dominatedPoints))
pp = np.array(list(paretoPoints))

print(pp.shape,dp.shape)
ax.scatter(dp[:,0],dp[:,1],dp[:,2])
ax.scatter(pp[:,0],pp[:,1],pp[:,2],color='green')


ax.set_xlabel('Rf')
ax.set_ylabel('Rp')
ax.set_zlabel('Rs')




import matplotlib.tri as mtri
triang = mtri.Triangulation(pp[:,0],pp[:,1])
ax.plot_trisurf(triang,pp[:,2],color='green',alpha=0.3)
plt.show()

In [ ]:
# CREATE DICTIONARY WITH RANKS AS KEY AND INDEX AS VALUE
def getConfs(points,orignaldf):
  point_confs = {}
  for i in range(0, len(orignaldf)):
    for val in points:
      if (list(val)==np.array(orignaldf[i][1:], dtype=np.float64).tolist()):
        point_confs[val]=orignaldf[i][0]

  return point_confs

In [ ]:
def getConfsSorted(confsDict):  
  # # summing all the values using sum()
  temp1 = {val: np.sum(float(idx) for idx in val) 
            for val, key in confsDict.items()} #sum the ranks of non dominated solution
        
  # using sorted to perform sorting as required
  temp2 = sorted(temp1.items(), key = lambda ele : temp1[ele[0]],reverse=False) #sort according to the sum before
  res={}
  for key, val in temp2:
    res[key]=confsDict[key]

  #print("The sorted dictionary : " + str(res))
  return res

In [ ]:
pareto = getConfs(paretoPoints, dimensionsAll)
pareto = getConfsSorted(pareto)
pareto

In [ ]:
dominated=getConfs(dominatedPoints, dimensionsAll)
dominated = getConfsSorted(dominated)
dominated